# **HỆ THỐNG GỢI Ý PHIM**

# **Giai đoạn 4: Xây dựng hệ thống gợi ý phim**

# **4.0. Dowload dữ liệu**

In [1]:
!wget https://media.githubusercontent.com/media/HoangNguyen31/Movie-Recommender-System/main/data/TMDB_movie_dataset.csv

--2024-03-07 18:38:11--  https://media.githubusercontent.com/media/HoangNguyen31/Movie-Recommender-System/main/data/TMDB_movie_dataset.csv
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 460614114 (439M) [text/plain]
Saving to: ‘TMDB_movie_dataset.csv’

TMDB_movie_dataset. 100%[===================>] 439.28M  98.5MB/s    in 4.2s    

2024-03-07 18:38:30 (104 MB/s) - ‘TMDB_movie_dataset.csv’ saved [460614114/460614114]



# **4.1. Import thư viện**

In [2]:
# Thư viện xử lý dữ liệu
import numpy as np
import pandas as pd

from wordcloud import WordCloud

import time
from datetime import datetime

# Thư viện trực quan hóa dữ liệu
import matplotlib.pyplot as plt
import seaborn as sns

# Thư viện học máy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from skimage import io

# Bỏ qua các warnings
import warnings
warnings.filterwarnings("ignore")

# **4.2. Đọc dữ liệu**

In [3]:
df = pd.read_csv("TMDB_movie_dataset.csv")
df["release_date"] = pd.to_datetime(df["release_date"])
df.head()

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,original_language,original_title,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,...,en,Inception,"Cobb, a skilled thief who commits corporate es...",83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili"
1,157336,Interstellar,8.417,32571,Released,2014-11-05,701729206,169,False,/pbrkL804c8yAv3zBZR4QPEafpAR.jpg,...,en,Interstellar,The adventures of a group of explorers who mak...,140.241,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America",English
2,155,The Dark Knight,8.512,30619,Released,2008-07-16,1004558444,152,False,/nMKdUUepR0i5zn0y1T4CsSB5chy.jpg,...,en,The Dark Knight,Batman raises the stakes in his war on crime. ...,130.643,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,Welcome to a world without rules.,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America","English, Mandarin"
3,19995,Avatar,7.573,29815,Released,2009-12-15,2923706026,162,False,/vL5LR6WdxWPjLPFRLe133jXWsh5.jpg,...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",79.932,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,Enter the world of Pandora.,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom","English, Spanish"
4,24428,The Avengers,7.710,29166,Released,2012-04-25,1518815515,143,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,...,en,The Avengers,When an unexpected enemy emerges and threatens...,98.082,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,Some assembly required.,"Science Fiction, Action, Adventure",Marvel Studios,United States of America,"English, Hindi, Russian"


# **4.3. Gợi ý phim dựa trên đánh giá trung bình**

### **Lọc những thuộc tính cần dùng**

In [4]:
columns = ["id", "title", "release_date", "vote_average", "vote_count", "poster_path",
           "original_language", "overview", "tagline", "genres", "production_companies",
           "production_countries", "spoken_languages"]
df_filtered = df[columns]
df_filtered = df_filtered.fillna("")

### **Tính điểm đánh giá cho bộ phim**
- Ta sẽ sử dụng điểm đánh giá cho mỗi bộ phim để sắp xếp các phim theo thứ tự đánh giá cao nhất đến thấp nhất
- Ở đây, để sắp xếp các bộ phim theo đánh giá, ta không thể sử dụng giá trị `vote_average` để đảm bảo tính công bằng (Ví dụ một bộ phim có điểm rất cao nhưng số vote lại rất ít sẽ không thể so sánh với một bộ phim có điểm thấp hơn nhưng số vote rất cao).
- Do đó, ta sẽ sử dụng weighted rating (điểm đánh giá có trọng số) được tính như sau:
$$\text{Weighted Rating: }wr = \left(\dfrac{v}{v+m}\times R\right) + \left(\dfrac{m}{v+m}\times C\right)$$

  Trong đó:
  - $v$: số lượng vote `vote_count`
  - $m$: số lượng vote ít nhất cần thiết để có thể tính $wr$.
  - $R$: điểm vote trung bình `vote_average`.
  - $C$: điểm vote trung bình toàn bộ dataset.

In [5]:
C = df_filtered.loc[:, "vote_average"].mean()
m = df_filtered.loc[:, "vote_count"].quantile(0.98)

In [6]:
def get_best_rating_movie(year=None, country=None):
  if (year is None) and (country is None):
    best_rating_movie = df_filtered.loc[df_filtered["vote_count"] >= m]
  elif (year is None) & (country is not None):
    best_rating_movie = df_filtered.loc[(df_filtered["production_countries"].str.contains(country)) &
                                        (df_filtered["vote_count"] >= m)]
  elif (year is not None) & (country is None):
    best_rating_movie = df_filtered.loc[(df_filtered["release_date"].dt.year == int(year)) &
                                        (df_filtered["vote_count"] >= m)]

  contribution = (best_rating_movie["vote_count"] / (best_rating_movie["vote_count"] + m) * best_rating_movie["vote_average"])
  adjustment = (m / (best_rating_movie["vote_count"] + m) * C)
  best_rating_movie["weighted_vote"] =  contribution + adjustment
  best_rating_movie = best_rating_movie.sort_values("weighted_vote", ascending=False)
  return best_rating_movie

### **Gợi ý phim dựa trên điểm**

In [7]:
def show_best_rating_movie(movie=5, year=None, country=None):
  best_rating_movie = get_best_rating_movie(year, country)
  fig, ax = plt.subplots(1, movie, figsize=(25,10))
  ax = ax.flatten()
  for i in range(movie):
    ax[i].axis("off")
    ax[i].set_title(best_rating_movie.iloc[i].title, fontsize=20)
    poster_path = io.imread("https://image.tmdb.org/t/p/w500/" + best_rating_movie.iloc[i].poster_path)
    ax[i].imshow(poster_path)
  fig.tight_layout()
  if (year is None) and (country is None):
    fig.suptitle(f"Top {movie} phim hay nhất mọi thời đại", fontsize=24)
  elif (year is None) & (country is not None):
    fig.suptitle(f"Top {movie} phim hay nhất mọi thời đại của nước {country}", fontsize=24)
  elif (year is not None) & (country is None):
    fig.suptitle(f"Top {movie} phim hay nhất trong năm {year}", fontsize=24)
  fig.show()

**Gợi ý phim hay nhất mọi thời đại**

In [8]:
show_best_rating_movie()

Output hidden; open in https://colab.research.google.com to view.

**Gợi ý phim hay nhất trong năm `2029`**

In [9]:
show_best_rating_movie(year=2019)

Output hidden; open in https://colab.research.google.com to view.

**Gợi ý phim hay nhất mọi thời đại của đất nước `Thái Lan`**

In [10]:
show_best_rating_movie(country="Thailand")

Output hidden; open in https://colab.research.google.com to view.

# **4.4. Gợi ý phim dựa trên mức độ tương đồng với tìm kiếm**

## **Tạo thuộc tính tổng hợp `(summary)`**

In [11]:
df_filtered["summary"] = df_filtered["original_language"] + " " + \
                         df_filtered["spoken_languages"].apply(lambda x: " ".join(x.split(","))) + " " + \
                         df_filtered["overview"] + " " + \
                         df_filtered["tagline"] + " " + \
                         df_filtered["genres"].apply(lambda x: " ".join(x.split(","))) + " " + \
                         df_filtered["production_companies"].apply(lambda x: " ".join(x.split(","))) + " " + \
                         df_filtered["production_countries"].apply(lambda x: " ".join(x.split(",")))

## **Trích xuất đặc trưng sử dụng `TF - IDF`**
- `TF - IDF` viết tắt của Term Frequency - Inverse Document Frequency.
- **Giá trị `TF`:**
  - `TF` là tần suất xuất hiện của một từ trong một tập văn bản, một thước đo mức độ phổ biến của một từ trong văn bản đó.
  $$tf(t,d) = \dfrac{n_t}{\sum_k n_k}$$
  Trong đó:
    - $tf(t,d)$: là giá trị TF (Term Frequency) hay tần suất xuất hiện của từ $t$ trong tập văn bản $d$.
    - $n_t$: là số lần xuất hiện của từ $t$ trong tập văn bản.
    - $\sum_k n_k$: là tổng số từ trong tập văn bản.

  - Giá trị `TF`: Xác định tần suất sử dụng/xuất hiện của một từ, giá trị `TF` càng cao cho thấy từ đó xuất hiện nhiều lần trong tài liệu, đồng nghĩa với việc từ đó quan trọng trong văn bản đó.

- **Giá trị `IDF`:**
  - `IDF` là một chỉ số dùng để đánh giá mức độ quan trọng của một từ trong một tập hợp văn bản.
  $$idf(t, D) = \log\dfrac{N}{|\{d\in D:t\in d\}|}$$
  Trong đó:
    - $N$: tổng số văn bản trong tập văn bản (tổng số chuỗi trong ngữ liệu - corpus) và $N=|D|$.
    - $|\{d\in D:t\in d\}|$: số lượng các văn bản (chuỗi) mà có sự xuất hiện của  từ $t$ và $d$ là một văn bản (chuỗi) trong tập các văn bản (ngữ liệu) $D$.
    - Khi một từ không xuất hiện trong tập văn bản, thì sẽ xuất hiện trường hợp phép chia cho 0 (division-by-zero), do đó, ta có thể căn chỉnh mẫu thành: $|\{d\in D:t\in d\}|+1$.

  - Giá trị `IDF`: Giá trị `IDF` cao cho thấy từ đó không phổ biến trong toàn bộ tập dữ liệu, và do đó, nó có khả năng đặc biệt và quan trọng.


- Khi đó, `TF - IDF` là việc kết hợp cả hai chỉ số trên: `TF` kết hợp với `IDF` tạo ra giá trị `TF - IDF` đánh giá được mức độ quan trọng của một từ không chỉ trong ngữ cảnh của một văn bản mà còn dựa trên độ hiếm của từ đó trong toàn bộ tập văn bản. Sự kết hợp của `TF` và `IDF` trong `TF - IDF` giúp cải thiện hiệu suất các tác vụ xử lý văn bản như tìm kiếm thông tin, phân loại, gợi ý và phân tích văn bản bằng cách đánh giá mức độ quan trọng của từng từ dựa trên tài liệu cụ thể và tập dữ liệu tổng thể. $$\text{tf-idf} = tf\times idf$$

In [12]:
tfidf = TfidfVectorizer(stop_words="english")
tfidf_matrix = tfidf.fit_transform(df_filtered["summary"])

## **Gợi ý phim**

In [13]:
def get_recommendations(title):
  idx = df_filtered.index[df_filtered["title"] == title][0]
  if df_filtered.loc[idx, "poster_path"] != " " and df_filtered.loc[idx, "poster_path"] is not None:
    poster = io.imread(f"https://image.tmdb.org/t/p/w500/{df_filtered.loc[idx, 'poster_path']}")
    plt.imshow(poster)
    plt.axis("off")
    plt.title(title, fontsize=8)
    plt.show()

    sim_scores = list(enumerate(cosine_similarity(tfidf_matrix, tfidf_matrix[idx])))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:9]

    movie_indices = [i[0] for i in sim_scores]
    result = df_filtered.iloc[movie_indices]

    print("\nCó thể bạn thích xem:")
    fig, ax = plt.subplots(2, 4, figsize=(25,10))
    ax = ax.flatten()
    for i in range(8):
      try:
        ax[i].axis("off")
        ax[i].set_title(result.iloc[i]["title"], fontsize=20)
        a = io.imread(f"https://image.tmdb.org/t/p/w500/" + result.iloc[i].poster_path)
        ax[i].imshow(a)
      except:
        ax[i].axis("off")
        ax[i].set_title(result.iloc[i]["title"], fontsize=20)
        a = io.imread(f"https://www.themoviedb.org/t/p/original/uc4RAVW1T3T29h6OQdr7zu4Blui.jpg")
        ax[i].imshow(a)

  fig.tight_layout()
  fig.show()

In [14]:
get_recommendations("The Dark Knight")

Output hidden; open in https://colab.research.google.com to view.